<a href="https://colab.research.google.com/github/muhdrizach/Human_Activity_Recognition_AI_model/blob/main/har.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import necessary libraries
import os
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, TimeDistributed, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Parameters
IMG_SIZE = 64  # Resize frames to 64x64
FRAME_COUNT = 30  # Number of frames per video
DATASET_PATH = "/content/drive/MyDrive/dataset"  # Path to your dataset in Google Drive
CLASSES = ["bikeride", "golf"]  # Define your activity classes

# Preprocess video frames with data augmentation (random horizontal flip)
def preprocess_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while len(frames) < FRAME_COUNT:
        ret, frame = cap.read()
        if not ret:
            break
        # Resize frame
        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        # Apply random horizontal flip
        if random.random() > 0.5:
            frame = cv2.flip(frame, 1)
        frames.append(frame)
    cap.release()
    # Pad with empty frames if fewer than FRAME_COUNT
    while len(frames) < FRAME_COUNT:
        frames.append(np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8))
    return np.array(frames)

In [ ]:
# Load dataset and preprocess all videos
def load_data(dataset_path, classes):
    X, y = [], []
    for label, activity in enumerate(classes):
        activity_path = os.path.join(dataset_path, activity)
        if not os.path.exists(activity_path):
            print(f"Warning: Folder {activity_path} does not exist.")
            continue
        for video_file in os.listdir(activity_path):
            video_path = os.path.join(activity_path, video_file)
            frames = preprocess_video(video_path)
            X.append(frames)
            y.append(label)
    return np.array(X), np.array(y)

In [ ]:
X, y = load_data(DATASET_PATH, CLASSES)

In [ ]:
# Normalize the dataset and one-hot encode labels
X = X / 255.0
y = to_categorical(y, num_classes=len(CLASSES))

In [ ]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train.argmax(axis=1)),
    y=y_train.argmax(axis=1)
)
class_weights = dict(enumerate(class_weights))

In [ ]:
# Define the model
model = Sequential([
    # TimeDistributed CNN for spatial feature extraction
    TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(FRAME_COUNT, IMG_SIZE, IMG_SIZE, 3)),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Flatten()),
    # Bidirectional LSTM for temporal feature extraction
    Bidirectional(LSTM(64, return_sequences=False)),
    Dense(128, activation='relu'),
    Dense(len(CLASSES), activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Display model summary
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed (TimeDistributed)   │ (None, 30, 62, 62, 32)      │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 30, 31, 31, 32)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_2 (TimeDistributed) │ (None, 30, 30752)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 128)                 │      15,778,304 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,795,970 (60.26 MB)

 Trainable params: 15,795,970 (60.26 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
# Train the model with class weights
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=16,
    class_weight=class_weights,
    callbacks=[early_stopping]
)

Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 359ms/step - accuracy: 0.6158 - loss: 0.6905 - val_accuracy: 0.7333 - val_loss: 0.5651
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 178ms/step - accuracy: 0.5247 - loss: 0.6907 - val_accuracy: 0.9000 - val_loss: 0.5196
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 177ms/step - accuracy: 0.8042 - loss: 0.5173 - val_accuracy: 0.8667 - val_loss: 0.4658
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step - accuracy: 0.8923 - loss: 0.4705 - val_accuracy: 0.9000 - val_loss: 0.3170
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step - accuracy: 0.9270 - loss: 0.3094 - val_accuracy: 0.9333 - val_loss: 0.2865
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step - accuracy: 0.9677 - loss: 0.2572 - val_accuracy: 0.9667 - val_loss: 0.1689
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 175ms/step - accuracy: 0.9870 - loss: 0.1254 - val_accuracy: 0.9667 - val_loss: 0.1220
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 170ms/step - accuracy: 0.9959 - loss: 0.0688 - val_accuracy: 0.9667 - val_loss

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.9667 - loss: 0.1220
Test Accuracy: 96.67%


In [ ]:
# Save the trained model to Google Drive
model.save("/content/drive/MyDrive/har_model.h5")
print("Model saved successfully.")

Model saved successfully.


In [ ]:
# Classification report and confusion matrix
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [ ]:
print("Classification Report:")
print(classification_report(y_true, y_pred_classes, target_names=CLASSES))

Classification Report:
              precision    recall  f1-score   support

    bikeride       0.94      1.00      0.97        17
        golf       1.00      0.92      0.96        13

    accuracy                           0.97        30
   macro avg       0.97      0.96      0.97        30
weighted avg       0.97      0.97      0.97        30



In [ ]:

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))

Confusion Matrix:
[[17  0]
 [ 1 12]]


In [ ]:
# Prediction function with error handling
def predict_activity(video_path):
    if not os.path.exists(video_path):
        print(f"Error: Video file does not exist at {video_path}")
        return "Error: File not found"

    frames = preprocess_video(video_path)
    frames = np.expand_dims(frames, axis=0)  # Add batch dimension
    prediction = model.predict(frames)
    print(f"Confidence Scores: {prediction}")
    predicted_class = np.argmax(prediction)
    return CLASSES[predicted_class]

In [ ]:
# Test the model with a sample video from Google Drive
sample_video_path = "/content/drive/MyDrive/dataset/testing_vedio/golf/testing_5.avi"  # Replace with the actual video path
predicted_activity = predict_activity(sample_video_path)
print(f"Predicted Activity: {predicted_activity}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Confidence Scores: [[0.02773046 0.9722695 ]]
Predicted Activity: golf
